In [42]:
# Ayesha Akhtar
# Deep Learning PA 2
import tensorflow as tf #tensorflow for cnn
%matplotlib inline
import os.path #filepath
import numpy as np #setting columns
import pandas as pd #dataframe
from pathlib import Path #path
from sklearn.model_selection import train_test_split #creating cnn

import tensorflow as tf
from sklearn.metrics import r2_score #r2 score

import csv
import pandas as pd

df = pd.read_csv('wiki_labels.csv') #open labels file
#del df['ID']
del df['dob']#remove irrelevant dob
del df['dob_str'] #remove irrelevant dob str
del df['photo_taken'] #remove irrelevant date
del df['gender'] #remove gender
del df['name'] #remove name
del df['face_location'] #remove face_location
del df['face_score'] #remove face_score
del df['second_face_score'] #remove

import zipfile
with zipfile.ZipFile('wiki_judge_images.zip', 'r') as zip_ref: #upload judge images as zip
    zip_ref.extractall("target") #open zip

with zipfile.ZipFile('wiki_labeled.zip', 'r') as zip_ref1: #upload labeled imgs as zip
    zip_ref1.extractall("ExtractLabeled") #open zip
agearr =[]
patharr =[]
idarr=[]
for x in df['age']:
    x = int(x);#turn ages into int
    agearr.append(x) #add to list
    
for x in df['ID']:
    idarr.append(x); #add IDs to list
    
for x in df['full_path']:
    x =x[1:-1]#remove unnecessary []
    x=x.strip("'") #remove quotes
    
    y = '/content/ExtractLabeled/wiki_labeled/'; #add this to path (Working on Colab Google)
    newstr = y+x; #add to path
    patharr.append(newstr) #save new path
  # /content/ExtractLabeled/wiki_labeled/00/10049200_1891-09-16_1958.png 
# for x in patharr:
#     print(x)
    
Paths = pd.Series(patharr, name="Paths").astype(str) #save all paths as string series
Ages = pd.Series(agearr, name="Ages").astype(np.int) #save all ages as int series

Concat = pd.concat([Paths, Ages], axis=1).sample(frac=1.0,random_state=1).reset_index(drop=True) #concat together
#print(Concat)
#print(df)
#ExtractLabeled\wiki_labeled\00\10049200_1891-0...
labeledPath = Path('content/ExtractLabeled/wiki_labeled') #get path for labeled images
LabelledPics = pd.Series(list(labeledPath.glob('**/*.png')), name ="LabelledPic") #find all pics in labeled images

#LabelledPics

img_df = Concat.sample(10000, random_state=1).reset_index(drop=True) #concat sample
train_df, test_df = train_test_split(img_df, train_size = 0.7,shuffle=True, random_state=1 ) #split train and test set

trainer = tf.keras.preprocessing.image.ImageDataGenerator( #train set image 
    rescale = 1./255, #resize image
    validation_split=0.2 
)
tester = tf.keras.preprocessing.image.ImageDataGenerator( #test set image
    rescale = 1./255 #resize
)

train_img = trainer.flow_from_dataframe( #get train images from df
    dataframe = train_df, #train df
    x_col = "Paths", #use paths as x
    y_col = "Ages", #use ages as y
    target_size = (120,120), #size
    color_mode = 'rgb', #color intact
    class_mode="raw", #class mode
    batch_size = 32, #batch size - tested with various
    shuffle=True, #shuffle around/randomize
    seed = 42,
    subset = "training"
)
validate = trainer.flow_from_dataframe( #validation imgs
    dataframe = train_df, #get from train ff
    x_col = "Paths", #use paths as x
    y_col = "Ages", #use ages as y
    target_size = (120,120), #size
    color_mode = 'rgb', #color intact intact
    class_mode="raw", #class
    batch_size = 32, #batch
    shuffle=True, #randomize
    seed = 42,
    subset = "validation"
)
test_img = tester.flow_from_dataframe( #test images
    dataframe = test_df, #use test df
    x_col = "Paths", #x column
    y_col = "Ages", #y column
    target_size = (120,120), #size
    color_mode = 'rgb', #color intact
    class_mode="raw", #class
    batch_size = 32,
    shuffle=False
)

inputs = tf.keras.Input(shape=(120,120,3)) #inputs shape
fLayer = tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu')(inputs) #layer 1 with 16 filters and relu
fLayer = tf.keras.layers.AveragePooling2D()(fLayer) #max pool/results

fLayer = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(fLayer) #layer 2 with 32 filters and relu
fLayer = tf.keras.layers.AveragePooling2D()(fLayer) #max pool/results
fLayer = tf.keras.layers.GlobalAveragePooling2D()(fLayer) #get average

fLayer = tf.keras.layers.Dense(64, activation='relu')(fLayer) #dense layer 1
fLayer = tf.keras.layers.Dense(64, activation='relu')(fLayer) #dense layer 2
fLayer = tf.keras.layers.Dense(64, activation='relu')(fLayer) #dense layer 3

output = tf.keras.layers.Dense(1, activation='linear')(fLayer) #get output in linear form
model = tf.keras.Model(inputs=inputs, outputs=output) #create model with inputs and outputs

model.compile( #model compiler
    optimizer='adam', #use adam
    loss = 'mse' #mse
)
modelResults = model.fit( #get model results fitted
    train_img,
    validation_data=validate,
    epochs=100, #100 epochs test
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience=5,
            restore_best_weights=True #get best possible weights
        )
    ]
)

predicted_age = np.squeeze(model.predict(test_img)) #test with predicted age on wiki label images
true_age = test_img.labels #get actual ages

rmse = np.sqrt(model.evaluate(test_img, verbose=0)) #determine rsme
print("RSME: ", rmse) #around 19
r2= r2_score(true_age,predicted_age) #get r2 score
print("R2: ", r2) #around 0



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


Found 5600 validated image filenames.
Found 1400 validated image filenames.
Found 3000 validated image filenames.
Epoch 1/100
175/175 [==============================] - 5s 28ms/step - loss: 1132.1778 - val_loss: 645.9075
Epoch 2/100
175/175 [==============================] - 5s 26ms/step - loss: 438.6340 - val_loss: 639.3805
Epoch 3/100
175/175 [==============================] - 5s 26ms/step - loss: 481.9100 - val_loss: 640.9985
Epoch 4/100
175/175 [==============================] - 5s 26ms/step - loss: 415.2037 - val_loss: 627.7242
Epoch 5/100
175/175 [==============================] - 4s 25ms/step - loss: 434.3864 - val_loss: 619.3061
Epoch 6/100
175/175 [==============================] - 5s 26ms/step - loss: 435.3664 - val_loss: 621.8293
Epoch 7/100
175/175 [==============================] - 4s 26ms/step - loss: 453.3080 - val_loss: 619.0695
Epoch 8/100
175/175 [==============================] - 4s 26ms/step - loss: 513.1293 - val_loss: 620.7501
Epoch 9/100
175/175 [================

In [39]:
model.save('./mymodel.h5') #save model for judge data

In [40]:
model = tf.keras.models.load_model('./mymodel.h5') #load model

In [41]:
mynewset = tf.keras.preprocessing.image.ImageDataGenerator( #test set image 
    rescale = 1./255, #resize image
    validation_split=0.2 
)

df1 = pd.read_csv('wiki_judge.csv') #read in judge info
del df1 ['gender'] #remove irrelevant gender column
del df1['face_score'] #remove irrelevant face_score column
del df1['second_face_score'] #remove second_face_score

#/content/target/wiki_judge_images/1.png
myarr=[]
for x in df1['full_path']: #for all paths
  y = '/content/target/wiki_judge_images/' #add this to path due to working on Colab Google
  x =x[1:-1]#remove unnecessary []
  x=x.strip("'") #remove quotes
  newstr = y+x #combine strings
  myarr.append(newstr) #save new path
ID =[];
for x in df1['ID']: #for all IDs
  ID.append(x) #save id
e_dataframe = pd.DataFrame( #put IDS and paths into dataframe
    {'ID': ID,
     'Path': myarr,})


train = mynewset.flow_from_dataframe( #get test images from df
    dataframe = e_dataframe, #judge imgs df
    x_col = "Path", #use paths as x
    y_col = "ID", #use ID as y - this isn't processed so it's fine to have this here
    target_size = (120,120), #size
    color_mode = 'rgb', #color intact
    class_mode="raw", #class since we're dealing with ints
    batch_size = 32,
    shuffle=False
)
predicted_age = np.squeeze(model.predict(train)) #get all predicted ages

a_zip = zip(train.labels, predicted_age) #zip predicted ages with their appropiate IDs

zipped_list = list(a_zip) #turn zipped list into list
# for x in zipped_list:
#   print(x)
fields = ['ID', 'Age']
with open('submission.csv', 'w') as f: # open submission.csv and write in it      
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerow(fields) #headers
    write.writerows(zipped_list)# IDS and predicted ages

Found 1409 validated image filenames.


# New Section

# New Section